In [1]:
import import_ipynb
import sys
sys.path.append('C:\\taewon_project\\DTML_selfatt')
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from Stock_Dataset import StockDataset
import argparse
from AttLSTM import att_LSTM
from Transformer_Encoder import Transformer
import numpy as np
import time
from metric import metric_acc as ACC
from metric import metric_mcc as MCC
import matplotlib.pyplot as plt
import csv
import os
from loss_fn import Selective_Regularization
from Stock_datasets_csv import stock_csv_read

def train(att_LSTM,transformer,                                  ## Model
          att_LSTM_optimizer, transformer_optimizer,   ## Optimizer
          Partition, args):                                      ## Data, loss function, argument
    trainloader = DataLoader(Partition['train'],
                             batch_size = args.batch_size,
                             shuffle=False, drop_last=True)

    att_LSTM.train()
    transformer.train()

    train_loss = 0.0
    for j, data in enumerate(trainloader):
        data_out_list = []
        for i in range(len(data)):

            att_LSTM_optimizer.zero_grad()
            transformer_optimizer.zero_grad()
            
            x_input = data[i][0].to(args.device)
            
            if i == 1:
                true_y = data[i][1].squeeze().float().to(args.device)
            
            att_LSTM.hidden = [hidden.to(args.device) for hidden in att_LSTM.init_hidden()]
            
            # 'list' object has no attribute 'float'

            hidden_context, attn_weights = att_LSTM(x_input)

            data_out_list.append(hidden_context)

        index_output = data_out_list[0]  # torch.Size([128, 10])
        stock_output = data_out_list[1] # torch.Size([128, 10])
        
        # torch.Size([64, 10, 10]) batch seq fea
        Transformer_input = index_output* args.market_beta + stock_output
        
        
        output1,wp,bp = transformer(Transformer_input.transpose(0,1))


        # output_ = torch.where(output1 >= 0.5, 1.0, 0.0)
        # output_.requires_grad=True
        
        loss = args.loss_fn(output1, true_y, wp.squeeze(), bp)
        loss.backward()

        att_LSTM_optimizer.step() ## parameter 갱신
        transformer_optimizer.step()

        train_loss += loss.item()

    train_loss = train_loss / len(trainloader)
    return att_LSTM, transformer, train_loss


def validation(att_LSTM,transformer,
               partition, args):
    valloader = DataLoader(partition['val'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    att_LSTM.eval()
    transformer.eval()
    val_loss = 0.0

    with torch.no_grad():
        for j, data in enumerate(valloader):

            data_out_list = []
            for i in range(len(data)):
                
                x_input = data[i][0].to(args.device)

                if i == 1:
                    true_y = data[i][1].squeeze().float().to(args.device)
                    
                att_LSTM.hidden = [hidden.to(args.device) for hidden in att_LSTM.init_hidden()]


                hidden_context, attn_weights = att_LSTM(x_input)

                data_out_list.append(hidden_context)

            index_output = data_out_list[0]  # torch.Size([128, 10])
            stock_output = data_out_list[1]  # torch.Size([128, 10])


            Transformer_input = index_output* args.market_beta  + stock_output

            output1,wp,bp = transformer(Transformer_input.transpose(0,1))

            loss = args.loss_fn(output1, true_y, wp.squeeze(), bp)
            val_loss += loss.item()

        val_loss = val_loss / len(valloader)
        return att_LSTM, transformer, val_loss


def test(att_LSTM, transformer,
               partition, args):
    testloader = DataLoader(partition['test'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    att_LSTM.eval()
    transformer.eval()

    ACC_metric = 0.0
    MCC_metric = 0.0
    with torch.no_grad():
        for j, data in enumerate(testloader):

            data_out_list = []
            for i in range(len(data)):
                
                x_input = data[i][0].to(args.device)

                if i == 1:
                    true_y = data[i][1].squeeze().float().to(args.device)

                att_LSTM.hidden = [hidden.to(args.device) for hidden in att_LSTM.init_hidden()]


                hidden_context, attn_weights = att_LSTM(x_input)

                data_out_list.append(hidden_context)

            index_output = data_out_list[0]  # torch.Size([128, 10])
            stock_output = data_out_list[1]  # torch.Size([128, 10])


            Transformer_input = index_output * args.market_beta + stock_output

            output1,wp,bp = transformer(Transformer_input.transpose(0,1))

            output_ = torch.where(output1 >= 0.5, 1.0, 0.0)

            output_.requires_grad = True

            ACC_metric += ACC(output_, true_y)
            MCC_metric += MCC(output_, true_y)

        ACC_metric = ACC_metric / len(testloader)
        MCC_metric = MCC_metric / len(testloader)

        return ACC_metric, MCC_metric



def experiment(partition, args):
    att_LSTM = args.att_LSTM(args.input_dim, args.hid_dim, args.output_dim, args.num_layers, args.batch_size, args.dropout, args.use_bn, args.attention_head, args.attn_size, args.activation)
    transformer = args.transformer(args.trans_feature_size, args.trans_num_laysers, args.dropout, args.batch_size, args.x_frames, args.trans_nhead)
    
    att_LSTM.to(args.device)
    transformer.to(args.device)

    if args.optim == 'SGD':
        att_LSTM_optimizer = optim.SGD(att_LSTM.parameters(), lr=args.lr, weight_decay=args.l2)
        transformer_optimizer = optim.SGD(transformer.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        att_LSTM_optimizer = optim.RMSprop(att_LSTM.parameters(), lr=args.lr, weight_decay=args.l2)
        transformer_optimizer = optim.RMSprop(transformer.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        att_LSTM_optimizer = optim.Adam(att_LSTM.parameters(), lr=args.lr, weight_decay=args.l2)
        transformer_optimizer = optim.Adam(transformer.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')

    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    # ===================================== #
    for epoch in range(args.epoch):
        ts = time.time()
        att_LSTM, transformer, train_loss = train(att_LSTM, transformer,
                                                  att_LSTM_optimizer, transformer_optimizer,
                                                  partition, args)

        att_LSTM, transformer, val_loss = validation(att_LSTM, transformer, partition, args)

        te = time.time()

        ## 각 에폭마다 모델을 저장하기 위한 코드
        torch.save(att_LSTM.state_dict(), args.split_file_path + '\\' + str(epoch) +'_epoch' +'_att_LSTM' +'.pt')
        torch.save(transformer.state_dict(), args.split_file_path + '\\' + str(epoch) +'_epoch' +'_transformer' +'.pt')

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        print('Epoch {}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
              .format(epoch, train_loss, val_loss, te - ts))

    ## val_losses에서 가장 값이 최소인 위치를 저장함
    site_val_losses = val_losses.index(min(val_losses)) ## 10 epoch일 경우 0번째~9번째 까지로 나옴
    att_LSTM = args.att_LSTM(args.input_dim, args.hid_dim, args.output_dim, args.num_layers, args.batch_size, args.dropout, args.use_bn, args.attention_head, args.attn_size, args.activation)
    transformer = args.transformer(args.trans_feature_size, args.trans_num_laysers,args.dropout, args.batch_size, args.x_frames, args.trans_nhead)
    att_LSTM.to(args.device)
    transformer.to(args.device)

    att_LSTM.load_state_dict(torch.load(args.split_file_path + '\\' + str(site_val_losses) +'_epoch' +'_att_LSTM'+ '.pt'))
    transformer.load_state_dict(torch.load(args.split_file_path + '\\' + str(site_val_losses) + '_epoch' + '_transformer' + '.pt'))

    ACC,MCC = test(att_LSTM, transformer, partition, args)
    print('ACC: {}, MCC: {}'.format(ACC, MCC))

    with open(args.split_file_path + '\\'+ str(site_val_losses)+'Epoch_test_metric' +'.csv', 'w') as fd:
        print('ACC: {}, MCC: {}'.format(ACC, MCC), file=fd)

    result = {}

    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['ACC'] = ACC
    result['MCC'] = MCC

    return vars(args), result


importing Jupyter notebook from Stock_Dataset.ipynb
importing Jupyter notebook from AttLSTM.ipynb
importing Jupyter notebook from attention.ipynb
importing Jupyter notebook from Transformer_Encoder.ipynb
importing Jupyter notebook from metric.ipynb
importing Jupyter notebook from loss_fn.ipynb
importing Jupyter notebook from Stock_datasets_csv.ipynb


In [2]:
# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

# ========= experiment setting ========== #
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
args.save_file_path = "c:\\taewon_project\\DTML_selfatt\\results"


# ====== hyperparameter ======= #
args.batch_size = 64
args.x_frames = 10
args.y_frames = 1
args.input_dim = 10
args.output_dim = 1
args.dropout = 0.2
args.use_bn = True
args.loss_fn = Selective_Regularization  ## loss function for classification : cross entropy
args.optim = 'Adam'
args.lr = 0.0005
args.l2 = 0.00001 #?
args.epoch = 100
args.activation="ReLU"

# ============= model ================== #
args.att_LSTM = att_LSTM
args.transformer = Transformer

# ====== att_lstm hyperparameter ======= #
args.hid_dim = 10
args.attention_head = 1
args.attn_size = 10
args.num_layers = 1
args.decoder_x_frames = 1

# ====== transformer hyperparameter ======= #
args.trans_feature_size = 40
args.trans_num_laysers = 1
args.trans_nhead = 10
args.market_beta = 0.1



In [3]:
## 실행 파일
args.data_list = os.listdir("C:\\taewon_project\\DTML_selfatt\\data\\kdd17\\ourpped")


with open(args.save_file_path + '\\' + 'DTML_result_t.csv', 'w', encoding='utf-8', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(["model", "stock", "entire_exp_time",  "avg_test_ACC", "avg_test_ACC_std", "avg_test_MCC"])

    for data in args.data_list:
        
        stock = data.split('.')[0]

        est = time.time()
        setattr(args, 'symbol', stock)
        args.new_file_path = args.save_file_path + '\\' + "DTML_" + args.symbol
        os.makedirs(args.new_file_path)
        
        
        csv_read = stock_csv_read(data,args.x_frames,args.y_frames)
        split_data_list = csv_read.cv_split()
        
        ACC_cv = []
        for i, data in enumerate(split_data_list):
            args.split_file_path = args.new_file_path + "\\" + str(i) +"th_iter"
            os.makedirs(args.split_file_path)
            # 0번째에 index 1번째에 stock 1개가 input으로 들어감
            trainset = StockDataset(data[0])
            valset = StockDataset(data[1])
            testset = StockDataset(data[2])
        

            partition = {'train': trainset, 'val': valset, 'test': testset}


            setting, result = experiment(partition, args)
            eet = time.time()
            entire_exp_time = eet - est

            fig = plt.figure()
            plt.plot(result['train_losses'])
            plt.plot(result['val_losses'])
            plt.legend(['train_losses', 'val_losses'], fontsize=15)
            plt.xlabel('epoch', fontsize=15)
            plt.ylabel('loss', fontsize=15)
            plt.grid()
            plt.savefig(args.split_file_path + '\\' + str(args.symbol) + '_fig' + '.png')
            plt.close(fig)
            ACC_cv.append(result['ACC'])
            # csv파일에 기록하기
        ACC_cv_ar = np.array(ACC_cv)
        acc_avg = np.mean(ACC_cv_ar)
        acc_std = np.std(ACC_cv_ar)

        wr.writerow(["DTML", args.symbol, entire_exp_time, acc_avg, acc_std, result['MCC']])

Epoch 0, Loss(train/val) 4.88525/4.88016. Took 0.45 sec
Epoch 1, Loss(train/val) 4.87988/4.88194. Took 0.65 sec
Epoch 2, Loss(train/val) 4.87837/4.88276. Took 0.54 sec
Epoch 3, Loss(train/val) 4.87981/4.88282. Took 0.45 sec
Epoch 4, Loss(train/val) 4.87951/4.88265. Took 0.45 sec
Epoch 5, Loss(train/val) 4.87900/4.88284. Took 0.46 sec
Epoch 6, Loss(train/val) 4.87785/4.88348. Took 0.45 sec
Epoch 7, Loss(train/val) 4.87869/4.88289. Took 0.46 sec
Epoch 8, Loss(train/val) 4.87805/4.88292. Took 0.46 sec
Epoch 9, Loss(train/val) 4.87674/4.88355. Took 0.46 sec
Epoch 10, Loss(train/val) 4.87813/4.88428. Took 0.46 sec
Epoch 11, Loss(train/val) 4.87885/4.88477. Took 0.51 sec
Epoch 12, Loss(train/val) 4.87907/4.88400. Took 0.45 sec
Epoch 13, Loss(train/val) 4.87784/4.88347. Took 0.48 sec
Epoch 14, Loss(train/val) 4.87778/4.88441. Took 0.48 sec
Epoch 15, Loss(train/val) 4.87902/4.88504. Took 0.56 sec
Epoch 16, Loss(train/val) 4.87770/4.88537. Took 0.46 sec
Epoch 17, Loss(train/val) 4.87835/4.88469